**banner**

# Getting More Data

So far we have been using data that is already downloaded into this repository. But we might want to get newer, or older, data or more data about other teams or players. This notebook will use the code library [nba_api](https://github.com/swar/nba_api) to get data from [NBA.com](https://nba.com).

Unfortunately the nba_api library doesn't work in JupyterLite where you are currently running these notebooks, so you will need to open this in [Google Colab](https://colab.research.google.com/github/callysto/basketball-and-data-science/blob/main/content/06-getting-more-data.ipynb) or the [Callysto Hub](https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fbasketball-and-data-science&branch=main&subPath=content/06-getting-more-data.ipynb&depth=1).

The code cell below will demonstrate getting career data about a particular player.

In [ ]:
!pip install nba_api

In [ ]:
player_name = 'Pascal Siakam'

from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
player = players.find_players_by_full_name(player_name)[0]
career = playercareerstats.PlayerCareerStats(player_id = player['id'])
for i, df in enumerate(career.get_data_frames()):
    print('dataframe', i)
    display(df)

In [10]:
player_names = ['Pascal Siakam','Lebron James','Kobe Bryant','Stephen Curry']
player_careers = []

for p in player_names:
    player = players.find_players_by_full_name(p)[0]
    careers = playercareerstats.PlayerCareerStats(player_id=player['id'])
    

[{'id': 1627783,
  'full_name': 'Pascal Siakam',
  'first_name': 'Pascal',
  'last_name': 'Siakam',
  'is_active': True},
 {'id': 2544,
  'full_name': 'LeBron James',
  'first_name': 'LeBron',
  'last_name': 'James',
  'is_active': True},
 {'id': 977,
  'full_name': 'Kobe Bryant',
  'first_name': 'Kobe',
  'last_name': 'Bryant',
  'is_active': False},
 {'id': 201939,
  'full_name': 'Stephen Curry',
  'first_name': 'Stephen',
  'last_name': 'Curry',
  'is_active': True}]

Another endpoint that's available is the All Time leaders for each category. If we are interested in who has the most assists or points, we can use this endpoint to get the top 10 from each category. Each dataframe is for a particular statistic. For example, the first dataframe is for # of games played. You can get specific statistics by specifying the index of the dataframe.

In [ ]:
from nba_api.stats.endpoints import alltimeleadersgrids as atl

all_time = atl.AllTimeLeadersGrids()

for df in all_time.get_data_frames():
    print('ALL TIME', df.columns.values.tolist()[2])
    display(df)

#steals dataframe is at index number 3
all_time_steals = all_time.get_data_frames()[3]

**banner**